<a href="https://colab.research.google.com/github/emoceanographer/usdanutrients/blob/master/Protein_requirements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import re
import statistics
from operator import add
import numpy as np

import math
import scipy
from scipy import stats
from scipy.optimize import least_squares

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/Nutrition/'
pop_path = path + 'WPP2019_PopulationByAgeSex_Medium.csv'
#BMI_path = path + 'NCD_BMI_MEAN.csv'
BMI_path = path + 'NCD_RisC_Lancet_2017_BMI_age_standardised_country.csv'
height_path = path + 'heights_over_time.csv'
fao_ctry_path = path + 'fao_ctry_codes.csv'

# Calculate avg. body mass of adults for each country

Population: Dataset is WPP2019, Population by age and sex, medium; for each country; plan is to, for each (country, year), match demographic groups to the age_aa requirement pattern and calculate needed aa requirements

In [0]:
pop_df = pd.read_csv(pop_path, encoding='latin-1') # each country has year, age group's population
BMI_df = pd.read_csv(BMI_path, encoding='latin-1') # each country has a year and an associated mean
height_df = pd.read_csv(height_path)
fao_ctry_codes = pd.read_csv(fao_ctry_path)

In [35]:
BMI_df.head(0)

,Country/Region/World,ISO,Sex,Year,Mean BMI,Mean BMI lower 95% uncertainty interval,Mean BMI upper 95% uncertainty interval,Prevalence of BMI>=30 kg/m² (obesity),Prevalence of BMI>=30 kg/m² lower 95% uncertainty interval,Prevalence of BMI>=30 kg/m² upper 95% uncertainty interval,Prevalence of BMI>=35 kg/m² (severe obesity),Prevalence of BMI>=35 kg/m² lower 95% uncertainty interval,Prevalence of BMI>=35 kg/m² upper 95% uncertainty interval,Prevalence of BMI<18.5 kg/m² (underweight),Prevalence of BMI<18.5 kg/m² lower 95% uncertainty interval,Prevalence of BMI<18.5 kg/m² upper 95% uncertainty interval,Prevalence of BMI 18.5 kg/m² to <20 kg/m²,Prevalence of BMI 18.5 kg/m² to <20 kg/m² lower 95% uncertainty interval,Prevalence of BMI 18.5 kg/m² to <20 kg/m² upper 95% uncertainty interval,Prevalence of BMI 20 kg/m² to <25 kg/m²,Prevalence of BMI 20 kg/m² to <25 kg/m² lower 95% uncertainty interval,Prevalence of BMI 20 kg/m² to <25 kg/m² upper 95% uncertainty interval,Prevalence of BMI 25 kg/m² to <30 kg/m²,Prevalence of BMI 25 kg/m² to <30 kg/m² lower 95% uncertainty interval,Prevalence of BMI 25 kg/m² to <30 kg/m² upper 95% uncertainty interval,Prevalence of BMI 30 kg/m² to <35 kg/m²,Prevalence of BMI 30 kg/m² to <35 kg/m² lower 95% uncertainty interval,Prevalence of BMI 30 kg/m² to <35 kg/m² upper 95% uncertainty interval,Prevalence of BMI 35 kg/m² to <40 kg/m²,Prevalence of BMI 35 kg/m² to <40 kg/m² lower 95% uncertainty interval,Prevalence of BMI 35 kg/m² to <40 kg/m² upper 95% uncertainty interval,Prevalence of BMI >=40 kg/m²(morbid obesity),Prevalence of BMI >=40 kg/m² lower 95% uncertainty interval,Prevalence of BMI >=40 kg/m² upper 95% uncertainty interval


In [0]:
BMI_df = BMI_df[['Country/Region/World','ISO','Sex','Year','Mean BMI','Mean BMI lower 95% uncertainty interval','Mean BMI upper 95% uncertainty interval']]

In [40]:
BMI_df.head()

,Country/Region/World,ISO,Sex,Year,Mean BMI,Mean BMI lower 95% uncertainty interval,Mean BMI upper 95% uncertainty interval
0,Afghanistan,AFG,Men,1975,18.999440,16.331119,21.763077
1,Afghanistan,AFG,Men,1976,19.105518,16.500077,21.801109
2,Afghanistan,AFG,Men,1977,19.212080,16.646507,21.842021
3,Afghanistan,AFG,Men,1978,19.319053,16.786899,21.876516
4,Afghanistan,AFG,Men,1979,19.425952,16.950002,21.930634


In [0]:
def BMI_extract(df,yrs):
  """Gets the average and standard deviation for selected years for male and female BMI"""
  # Male
  temp = df[(df['Sex']=='Men') & (df['Year'].isin(yrs))]
  avgM = temp['Mean BMI'].mean(axis=0)
  stdM = ((temp['Mean BMI upper 95% uncertainty interval'] - temp['Mean BMI lower 95% uncertainty interval'])/4).mean(axis=0)
  
   # Female
  temp = df[(df['Sex']=='Women') & (df['Year'].isin(yrs))]
  avgF = temp['Mean BMI'].mean(axis=0)
  stdF = ((temp['Mean BMI upper 95% uncertainty interval'] - temp['Mean BMI lower 95% uncertainty interval'])/4).mean(axis=0)
  
  return avgM, stdM, avgF, stdF

In [0]:
yr_str = '2016'
country = 'Afghanistan'
BMI = BMI_df[BMI_df['Country/Region/World'] == country]

a,b,c,d = BMI_extract(BMI,[2011,2012,2013])

In [44]:
print([a,b,c,d])

[22.344502237636732, 1.1946179814958422, 23.723025716873337, 0.5270138711887494]


In [0]:
# Age group definitions
child_age_grps = ['0-4', '5-9', '10-14', '15-19']
adult_age_grps = ['20-24', '25-29', '30-34', '35-39',
       '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74',
       '75-79', '80-84', '85-89', '90-94', '95-99', '100+'] # age groups for pop. data

In [0]:
ctry_mismatches = {'China, Hong Kong SAR': ['none'], 'China, Macao SAR': ['none'], 
                   'China, Taiwan Province of China': ['none'], 
                   "Dem. People's Republic of Korea": "Democratic People's Republic of Korea",
                   'French Guiana': ['none'], 'French Polynesia': ['none'], 
                   'Guadeloupe': ['none'],'Martinique': ['none'],
                   'United Kingdom': ['United Kingdom of Great Britain and Northern Ireland']}

In [0]:
temp = re.search(r'\[\.(.*?)\]', BMI[yr_str + '.2'].values[0])
temp

In [0]:
# for each year and country, calculate the average mass of an adult
avg_weight = {}
for country in list(pop_df.Location.unique()): # cycle through each country
  # population
  temp = pop_df[pop_df['Location'].isin([country])]
  temp = temp[temp['AgeGrp'].isin(adult_age_grps)]
  
  # BMI
  ctr = 0
  if country in BMI_df['Country/Region'].unique(): # skip countries for which there is no BMI
    ctr = 1
    BMI = BMI_df[BMI_df['Country/Region'] == country]
    
  if country in ctry_mismatches: 
    if ctry_mismatches[country][0] in BMI_df['Country'].unique():
      ctr = 1
      BMI = BMI_df[BMI_df['Country/Region'] == ctry_mismatches[country][0]]
  
  if ctr == 1:

    if country == 'United Kingdom':
      ctry_code = 'GBR'
    else:
      ctry_code = fao_ctry_codes[fao_ctry_codes['Short name'] == country]['ISO3'].values[0]
    
    # Heights (gets people 20-40 in 2011-2013 by year of birth)
    hts = height_df[height_df['Code'] == ctry_code]
    hts_m = hts[(hts['Year']>1970) & (hts['Year']<1994)]['Males (centimeters)'].mean()
    hts_f = hts[(hts['Year']>1970) & (hts['Year']<1994)]['Females (centimeters)'].mean()
    
    f_pop_year = []
    m_pop_year = []
    f_BMI = []
    m_BMI = []
    
    for year in list(pop_df.Time.unique()):
      if year < 2014 and year > 2010: # get 2011-2013
        # population
        f_adult_yr_pop = sum(temp[temp['Time'].isin([year])]['PopFemale'])
        m_adult_yr_pop = sum(temp[temp['Time'].isin([year])]['PopMale'])
        f_pop_year.append(f_adult_yr_pop*1000) # units originally in thousands of people
        m_pop_year.append(m_adult_yr_pop*1000)

        # BMI (average only)
        yr_str = str(year)
        BMI_f_temp = re.search(r'^.*(?=(\[))', BMI[yr_str + '.2'].values[0])
        BMI_m_temp = re.search(r'^.*(?=(\[))', BMI[yr_str + '.1'].values[0])
        if BMI_f_temp:
          f_BMI.append(float(BMI_f_temp.group(0)))
        else:
          f_BMI.append(np.nan)
        if BMI_m_temp:
          m_BMI.append(float(BMI_m_temp.group(0)))
        else:
          m_BMI.append(np.nan)
    
    # Weight calculation (kg)
    f_weight = (hts_f/100)**2 * statistics.mean(f_BMI)
    m_weight = (hts_m/100)**2 * statistics.mean(m_BMI)
    
    # Population weighting
    avg_weight[country] = (f_weight*statistics.mean(f_pop_year) + m_weight*statistics.mean(m_pop_year)) / statistics.mean(list(map(add, f_pop_year, m_pop_year)))
  else:
    print(country)

Africa
Aruba
Asia
Australia/New Zealand
Caribbean
Central America
Central and Southern Asia
Central Asia
Channel Islands
China, Hong Kong SAR
China, Macao SAR
China, Taiwan Province of China
Curaçao
Dem. People's Republic of Korea
Eastern Africa
Eastern and South-Eastern Asia
Eastern Asia
Eastern Europe
Europe
Europe and Northern America
French Guiana
French Polynesia
Guadeloupe
Guam
High-income countries
Land-locked Developing Countries (LLDC)
Latin America and the Caribbean
Least developed countries
Less developed regions
Less developed regions, excluding China
Less developed regions, excluding least developed countries
Low-income countries
Lower-middle-income countries
Martinique
Mayotte
Melanesia
Micronesia
Micronesia (Fed. States of)
Middle Africa
Middle-income countries
More developed regions
New Caledonia
No income group available
North Macedonia
Northern Africa
Northern Africa and Western Asia
Northern America
Northern Europe
Oceania
Oceania (excluding Australia and New Zealand

In [0]:
ctry_weights = pd.DataFrame.from_dict(avg_weight,orient='index')
ctry_weights = ctry_weights.reset_index()
missing = ctry_weights[ctry_weights[0].isna()]
list(missing['index'].unique()) # should be just those with actually no data (e.g. Sudan)



['South Sudan', 'Sudan']

In [0]:
ctry_weights

,index,0
0,Afghanistan,59.128907
1,Albania,74.300504
2,Algeria,69.186634
3,Angola,61.446617
4,Antigua and Barbuda,72.731290
...,...,...
175,Venezuela (Bolivarian Republic of),71.597418
176,Viet Nam,53.435563
177,Yemen,59.061992
178,Zambia,59.035999


# Calculate AA requirements per capita (adults)

In [0]:
case = 'baseline'

if case == 'baseline':

  AA_adult_req = {'Histidine': 15, 'Isoleucine': 30, 'Leucine': 59, 'Lysine': 45, 
                 'SAA': 22, 'AAA': 38, 'Threonine': 23, 'Tryptophan': 6, 'Valine': 39} # mg AA /g protein
  AA_total = .66 # g/kg
elif case == 'safe':
  AA_adult_req = {'Histidine': 12, 'Isoleucine': 24, 'Leucine': 47, 'Lysine': 36, 
                 'SAA': 17, 'AAA': 30, 'Threonine': 18, 'Tryptophan': 5, 'Valine': 31} # mg AA /g protein
  AA_total = .83 # g/kg

In [0]:
SAA = ['Methionine', 'Cystine'] # there are others, but not in my diet AA list
AAA = ['Phenylalanine', 'Tyrosine'] # also tryptophan and histidine but they are listed separately

In [0]:
diet_AA = pd.read_csv(path + 'diet_aa_1126_waste.csv')

In [0]:
AA_ctry = {}
AA_all = {}
for country in diet_AA['Unnamed: 0'].unique().tolist():
  if country in ctry_weights['index'].unique().tolist():
    diet_ctry = diet_AA[diet_AA['Unnamed: 0'] == country]
    AA_ctry[country] = {}
    AA_all[country] = {}
    for AA in AA_adult_req: 
      avg_weight = ctry_weights[ctry_weights['index']==country][0].values[0] 
      AA_req =  avg_weight * AA_total * AA_adult_req[AA] * 0.001 #g/capita/day
      if AA == 'SAA':
        aa_diet = []
        for aa in SAA: 
          aa_diet.append(diet_ctry[aa].values[0])
        AA_diet = sum(aa_diet)
      elif AA == 'AAA':
        aa_diet = []
        for aa in AAA: 
          aa_diet.append(diet_ctry[aa].values[0])
        AA_diet = sum(aa_diet)
      else:
        AA_diet = diet_ctry[AA].values[0]
      AA_ctry[country][AA] = AA_diet / AA_req #[AA_req, AA_diet, AA_diet/AA_req] 
      AA_all[country][AA] = [AA_req, AA_diet, AA_diet/AA_req] 
      if AA_ctry[country][AA]<1:
        print(country, ' lacking ' , AA, ' see: ', AA_req, AA_diet)
    total_protein_req = avg_weight * AA_total
    total_protein_diet = diet_ctry['protein'].values[0]
    AA_ctry[country]['protein'] =  total_protein_diet/total_protein_req #[total_protein_req, total_protein_diet, total_protein_diet/total_protein_req]
    AA_all[country]['protein'] = [total_protein_req, total_protein_diet, total_protein_diet/total_protein_req]
    if AA_ctry[country]['protein'] < 1: 
      print(country, ' lacking total protein, see' , total_protein_req, total_protein_diet)

Bangladesh  lacking total protein, see 34.90687288366379 31.73755243612161
Benin  lacking total protein, see 41.16636438781105 35.81830660081956
Botswana  lacking total protein, see 44.571305692524426 43.23820221342434
Cameroon  lacking  Valine  see:  1.6929241316672778 1.5879065411599425
Cameroon  lacking total protein, see 43.40831106839174 34.05241593091996
Central African Republic  lacking total protein, see 39.838969339630815 33.65412431702833
Chad  lacking  Lysine  see:  1.8061477007940112 1.737731028701204
Chad  lacking total protein, see 40.136615573200245 36.96810560982088
Congo  lacking total protein, see 40.666855241580585 39.85403459190722
Côte d'Ivoire  lacking  Isoleucine  see:  1.2620946551076782 1.2330876736316525
Côte d'Ivoire  lacking  Leucine  see:  2.4821194883784337 2.3372100406177645
Côte d'Ivoire  lacking  Lysine  see:  1.8931419826615172 1.8191773029734015
Côte d'Ivoire  lacking  Valine  see:  1.6407230516399816 1.4668711035455846
Côte d'Ivoire  lacking total pr

In [0]:
AA_ratios = pd.DataFrame.from_dict(AA_ctry,orient='index')

In [0]:
savepath = path + 'AA_ratios_safe.csv'
AA_ratios.to_csv(savepath)

In [0]:
AA_all['Afghanistan'] # requirement, diet, ratio

{'AAA': [1.4829529909295607, 3.181650213947502, 2.1454828530695],
 'Histidine': [0.5853761806300898, 1.0671316584469748, 1.8229844222536198],
 'Isoleucine': [1.1707523612601796, 1.672272505887443, 1.4283742328629045],
 'Leucine': [2.3024796438116866, 3.1164248209910137, 1.353508088276457],
 'Lysine': [1.7561285418902695, 2.0686111846277617, 1.1779383657195963],
 'SAA': [0.8585517315907983, 1.52602662708826, 1.7774428388382688],
 'Threonine': [0.897576810299471, 1.3413229154862352, 1.494382319256567],
 'Tryptophan': [0.23415047225203592, 0.4871218714984065, 2.080379624321561],
 'Valine': [1.5219780696382335, 1.9935380541448853, 1.3098336263273092],
 'protein': [39.02507870867265, 50.42202361446346, 1.2920415610400302]}

## Undernourishment
Use the average diet as the mean of lognormal & the avg. requirement as threshold for undernourishment (NB FAO would use a 5th percentile for this?)

In [0]:
def prop_under(s_mean, CV, x):
  sigma = ((math.log(CV**2+1)))**0.5
  mu = math.log(s_mean) - sigma**2/2
  CDF = 1/2 + 1/2*scipy.special.erf((math.log(x) - mu)/(2**0.5*sigma))
  return CDF

In [0]:
variable_diet = {}
for country in AA_all:
  variable_diet[country] = {}
  for AA in AA_all[country]:
    pr_mean = AA_all[country][AA][1] # average amount in diet
    CV = 0.25 # sample

    req = AA_all[country][AA][0] # average requirement

    CDF = prop_under(pr_mean, sigma, req)
    variable_diet[country][AA] = CDF

In [0]:
variable_diet_df = pd.DataFrame.from_dict(variable_diet,orient='index')
variable_diet_df.head()

,Histidine,Isoleucine,Leucine,Lysine,SAA,AAA,Threonine,Tryptophan,Valine,protein
Afghanistan,0.159174,0.310610,0.350879,0.461741,0.172173,0.091065,0.278464,0.101937,0.376299,0.387066
Albania,0.017260,0.032140,0.048495,0.022688,0.023415,0.005792,0.017214,0.018024,0.052496,0.142396
Algeria,0.100112,0.142358,0.179787,0.182758,0.094143,0.035596,0.110453,0.067432,0.189942,0.281223
Angola,0.136782,0.311888,0.352272,0.232458,0.203201,0.116810,0.211833,0.186711,0.416354,0.553057
Antigua and Barbuda,0.014032,0.040017,0.063740,0.016608,0.021050,0.009133,0.018269,0.016499,0.077025,0.171813


In [0]:
savepath = path + 'undernourished_25.csv'
variable_diet_df.to_csv(savepath)

# Old Code

In [0]:
# BMI regex work! (save me for now!)
temp = BMI['2016'].values[0]
print(temp)
m = re.search(r"[^[]*\[([^]]*)\]", temp).groups()[0] # gets the things in the brackets!
n = re.search(r'^.*(?=(\[))', temp).group(0)
print(m)
print(n)

23.4 [22.0-24.8]
22.0-24.8
23.4 


In [0]:
adult_weight = 136.68 # average adult weight


In [0]:
# calculate average height (m) for each country; look into this later
ctry_hts = {}
for country in list(pop_df.Location.unique()): 
  if country in list(height_df.Entity.unique()):
    ht = height_df[height_df['Entity'].isin([country])]
    m_ht = statistics.mean(ht['Males (centimeters)'])
    f_ht = statistics.mean(ht['Females (centimeters)'])
    ctry_hts[country] = {'M': m_ht/100, 'F': f_ht/100}

In [0]:
# match those historical populations with average weight
avg_adult_weight = {}
total_adult_weight = {}

for country in list(pop_df.Location.unique()): 
  BMI = BMI_df[BMI_df['Country'].isin([country])]
  f_yrs_weight = {}
  m_yrs_weight = {}
  
  ctry_ht = ctry_hts.get(country)
  if ctry_ht:
    m_ht = ctry_ht['M'] # get the average height for that country
    f_ht = ctry_ht['F']

    f_tot_weight = {} # pre-allocate
    m_tot_weight = {}
    tot_yr_weight = {}
    for year in list(pop_df.Time.unique()):
      # BMI
      if str(year) in BMI.columns: # only go through years for which we have data
        avg_m_BMI = re.sub(r'\[[^]]*\]','',BMI[str(year)+'.1'].item())
        avg_f_BMI = re.sub(r'\[[^]]*\]','',BMI[str(year)+'.2'].item())
        
        f_yrs_weight[year] = float(avg_f_BMI)*(f_ht)**2 # calculates weight from BMI and height
        m_yrs_weight[year] = float(avg_m_BMI)*(m_ht)**2
        
        f_tot_weight[year] = f_adult_population[country][year]*f_yrs_weight[year]
        m_tot_weight[year] = m_adult_population[country][year]*m_yrs_weight[year]
        tot_yr_weight[year] = f_tot_weight[year] + m_tot_weight[year]
        
    avg_adult_weight[country] = {'f': f_yrs_weight, 'm': m_yrs_weight}
  
    total_adult_weight[country] = tot_yr_weight
  else:
    print([country, ' did not have associated height'])

['Africa', ' did not have associated height']
['Aruba', ' did not have associated height']
['Asia', ' did not have associated height']
['Australia/New Zealand', ' did not have associated height']
['Bolivia (Plurinational State of)', ' did not have associated height']
['Brunei Darussalam', ' did not have associated height']
['Cabo Verde', ' did not have associated height']
['Caribbean', ' did not have associated height']
['Central America', ' did not have associated height']
['Central and Southern Asia', ' did not have associated height']
['Central Asia', ' did not have associated height']
['Channel Islands', ' did not have associated height']
['China, Hong Kong SAR', ' did not have associated height']
['China, Macao SAR', ' did not have associated height']
['China, Taiwan Province of China', ' did not have associated height']
["Côte d'Ivoire", ' did not have associated height']
['Curaçao', ' did not have associated height']
['Czechia', ' did not have associated height']
["Dem. People's

ValueError: ignored